

#   ПИ21-1В
#   Сабиров А.А.



---
# Импортируем необходимые библиотеки

In [ ]:
%pip install pymorphy3

In [ ]:
%pip install rarfile

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import matplotlib.pyplot as plt

from collections import Counter, defaultdict
from itertools import chain

import multiprocessing
from tqdm import tqdm
from time import perf_counter

from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer

import pymorphy3


from wordcloud import WordCloud

import dask
import dask.bag as db
from dask.diagnostics import ProgressBar

In [ ]:
import os
import glob

import subprocess

import zipfile
from rarfile import RarFile


os.getcwd()

'/content'



---
## Смотрим csv-файл с книгами



In [ ]:
books = pd.read_csv("https://raw.githubusercontent.com/ibbvs/jasa/main/TOBD-labs/data%20sources/libr.csv", encoding="utf-8", sep=";", on_bad_lines="skip", names=["Источник", "Название", "Жанр", "Артикль", "Тег", 1, 0])
books["Тег"] = books["Тег"].apply(lambda x: x.lower())

books.head(3)

,Источник,Название,Жанр,Артикль,Тег,1,0
0,Anekdot.ru,Лучшие афоризмы и стишки 1997-1999 гг.,юмор,464500,aforizm,1,0
1,Anekdot.ru,Салон 1-67: Сборник любительских околоюморных текстов,юмор,569165,salon,1,0
2,Anekdot.ru,Самые популярные истории XX века,юмор,2643776,story979,1,0


In [ ]:
books[books["Жанр"] == "история"]

,Источник,Название,Жанр,Артикль,Тег,1,0
5,DLIFL Center,Краткая история России и СС (американский взгляд),история,172601,rhistory,1,0
74,Акунин Борис,Пелагея и белый бульдог,история,498131,pelagya,1,0
76,Акунин Борис,Турецкий гамбит,история,359994,gambit,1,0
77,Аладырев Святослав,Извек,история,808067,izvek,1,0
101,Алешкин Тимофей,Рассказы,история,106591,aleshkin,1,0
...,...,...,...,...,...,...,...
3796,NaN,Мифы индейцев Южной Америки (для взрослых),история,481162,indian,1,0
3797,NaN,Мрамор,история,120638,br_marmr,1,0
3851,NaN,Рапорты русских военачальников о Бородинском сражении,история,179239,raport,1,0
3887,NaN,Слово о полку Игореве,история,56639,igor,1,0


In [ ]:
books[books["Источник"] == "Акунин Борис"]

,Источник,Название,Жанр,Артикль,Тег,1,0
71,Акунин Борис,Азазель,драма,398832,azazel,1,0
72,Акунин Борис,Алтын-Толобас,драма,663917,akunin1,1,0
73,Акунин Борис,Декоратор,драма,313515,dekorat,1,1
74,Акунин Борис,Пелагея и белый бульдог,история,498131,pelagya,1,0
75,Акунин Борис,Пелагия и белый бульдог,драма,495044,pelagiya,1,0
76,Акунин Борис,Турецкий гамбит,история,359994,gambit,1,0
4007,Акунин Борис,Лефиафан,детектив,395527,leviafan,1,0
4008,Акунин Борис,Любовница смерти,детектив,446932,lub,1,1
4009,Акунин Борис,Пиковый валет,детектив,252148,valet,1,1


---
## Работа с файловой системой
*   Извлекаем файлы из zip-архива
*   Приводим полученные файлы к txt-формату

In [ ]:
# # Указываем путь к ZIP-файлу
# zip_path = "drive/MyDrive/HISTORY.zip"

# # Открываем ZIP-файл в режиме чтения
# with zipfile.ZipFile(zip_path, "r") as zip_ref:
#     # Проходим по всем файлам в архиве
#     for file_name in zip_ref.namelist():
#         # Извлекаем файл из архива
#         zip_ref.extract(file_name, "/content")

In [ ]:
# input_folder = "HISTORY"
# output_folder = "drive/MyDrive/HISTORY-TXT"
# imp_exe = "drive/MyDrive/imp.exe"

# # Получаем список файлов в папке HISTORY
# imp_files = [f for f in os.listdir(input_folder) if f.endswith(".imp")]


# for imp_file in imp_files:
#     imp_path = os.path.join(input_folder, imp_file)



#     # Выполняем команду imp.exe с использованием subprocess
#     command = [imp_exe, "e", imp_path, "-o" + output_folder]
#     subprocess.run(command, check=True)




# rar_files = [f for f in os.listdir(input_folder) if f.endswith(".rar")]

# for rar_file in rar_files:
#     rar_path = os.path.join(input_folder, rar_file)

#     with RarFile(rar_path, 'r') as rf:
#         # Извлекаем все файлы из архива в тот же каталог
#         rf.extractall(path=output_folder)

---
# Получаем список с путями к файлам, для последующей передачи к dask.bag

In [ ]:
directory_path = "drive/MyDrive/HISTORY-TXT"
files = sorted(glob.glob(os.path.join(directory_path, "*")))


files[:5], len(files)

(['drive/MyDrive/HISTORY-TXT/58.TXT',
  'drive/MyDrive/HISTORY-TXT/ADID1R.TXT',
  'drive/MyDrive/HISTORY-TXT/AFANEOR.TXT',
  'drive/MyDrive/HISTORY-TXT/ALESHK01.TXT',
  'drive/MyDrive/HISTORY-TXT/ALESHKIN.TXT'],
 294)

In [ ]:
file_names = list(map(lambda x: x.split("/")[-1][:-4].lower(), files))

print(file_names[:10])

['58', 'adid1r', 'afaneor', 'aleshk01', 'aleshkin', 'apian', 'arabsk', 'aspasia', 'augusts', 'a_gili1']


In [ ]:
a = set(books[books["Жанр"] == "история"]["Тег"].values)
b = set(file_names)


unique = list(a ^ b)


# igra3 - пустой файл, legend - белорусский рассказ "Седая Легенда", такого нет в csv-файле
print(f"Книг с такими тегами нет в csv-файле: {unique}")

Книг с такими тегами нет в csv-файле: ['legend', 'igra3']


---
# Блок с функциями для работы по всему тексту:

*   Очистка от знаков пунктуации, цифр, спец. символов
*   Инициализация и дальнейшее удаление стоп-слов
*   Стемминг, Лемматизация
*   Функции для подсчета расстояния Левенштейна

---
## Инициализация необходимых переменных

In [ ]:
#ИНИЦИАЛИЗАЦИЯ СПИСКА РУССКИХ СТОП-СЛОВ
SWrus = nltk.corpus.stopwords.words('russian')

# Оставляю слова длиной больше 1
SWrus = [word for word in SWrus if len(word) > 1]

SWrus[:5], SWrus[-5:]

(['во', 'не', 'что', 'он', 'на'],
 ['более', 'всегда', 'конечно', 'всю', 'между'])

In [ ]:
# ИНИЦИАЛИЗАЦИЯ ИНСТРУМЕНТОВ ДЛЯ СТЕММИНГА И ЛЕММАТИЗАЦИИ
stemmer = SnowballStemmer("russian")
lemmer = pymorphy3.MorphAnalyzer()



def lemmatize_word(word):
    return lemmer.parse(word)[0].normal_form

def stem_word(word):
    return stemmer.stem(word)

In [ ]:
# ФУНКЦИЯ КОТОРАЯ СОБИРАЕТ ВСЕ СТРОКИ В ОДНУ
# Применять через map_partitions
flatten_rows = lambda rows: ''.join(rows).strip()


# ФУНКЦИЯ СОЕДИНЯЕТ СЛОВА МЕЖДУ КОТОРЫМИ ЕСТЬ ЗНАК "-", ТАК КАК ЭТО ОБЫЧНО ЗНАК ПЕРЕНОСА И ЕСЛИ ЕГО ПРОСТО УДАЛИТЬ, ТЕРЯЕТСЯ СЛОВО
remove_dash = lambda rows: re.sub(r"-\n", "", rows)


#ФУНКЦИЯ ДЛЯ ОЧИСТКИ СТРОК ОТ ЗНАКОВ ПУНКТУАЦИИ, ЦИФР, СПЕЦ.СИМВОЛОВ И АНГЛИЙСКИХ СЛОВ
remove_punct = lambda rows: re.sub(r"[^а-яА-ЯёЁ]", " ", rows.lower()) # r"[\W+|\d]+"


# ОТДЕЛЬНАЯ ФУНКЦИЯ - УДАЛЯЕТ ЛЮБЫЕ ОДИНОЧНЫЕ СИМВОЛЫ
remove_alone = lambda rows: re.sub(r"\b[^я]\b", " ", rows)


# ФУНКЦИЯ ДЛЯ УДАЛЕНИЯ СТОП-СЛОВ ИЗ СТРОКИ
remove_stopwords = lambda rows, stop_words: re.sub(r"\b(?:{})\b|\b\w\b".format("|".join(map(re.escape, stop_words))), "", rows)


# ФУНКЦИЯ КОТОРАЯ РАЗБИВАЕТ ВЕСЬ ТЕКСТ НА СЛОВА
def tokenize_word(rows):
    if isinstance(rows, str):
        return re.findall(r"\b\w+\b", rows)
    return rows

---
# Применяем функции выше к нашему Bag-у
### Перед этим сделаем копию у которой не будем удалять стоп-слова

In [ ]:
# Взял 50 первых книг, если взять все, получается (из 234 книг) больше 20_000_000 миллионов слов с учетом стоп-слов и 15_000_000 очищенных от стоп-слов
# По времени неплохо так обрабатывает


b = db.read_text(files, encoding='windows-1251')

b

dask.bag<bag-from-delayed, npartitions=294>

In [ ]:
# Сохраняем копию bag с стоп-словами для дальнейшего сравнения
b_copy = b.map(lambda x: x)

b_copy

dask.bag<lambda, npartitions=294>

In [ ]:
# 1
b = b.map_partitions(flatten_rows)
b_copy = b_copy.map_partitions(flatten_rows)


# 2
b = b.map_partitions(remove_dash)
b_copy = b_copy.map_partitions(remove_dash)


# 3
b = b.map_partitions(remove_punct)
b_copy = b_copy.map_partitions(remove_punct)


# 4
b = b.map_partitions(remove_alone)
b_copy = b_copy.map_partitions(remove_alone)


# 5
b = b.map_partitions(remove_stopwords, stop_words=SWrus)


# # 6
b = b.map_partitions(tokenize_word)
b_copy = b_copy.map_partitions(tokenize_word)



---
## Немного отладки, смотрим первую партицию - что в ней находится после применения функций

In [ ]:
# Получаем все элементы первой партиции - текст по файлу 58.TXT
partition_data_without = dask.compute(b.to_delayed()[0])[0]
partition_data_with = dask.compute(b_copy.to_delayed()[0])[0]


print(partition_data_without[:15], partition_data_with[:15], sep="\n")

['фрин', 'москва', 'подольск', 'москва', 'отличие', 'большинства', 'моих', 'близких', 'друзей', 'особенно', 'подруг', 'человек', 'толстокожий', 'малочувствительной', 'нервной']
['фрин', 'москва', 'подольск', 'москва', 'отличие', 'от', 'большинства', 'моих', 'близких', 'друзей', 'особенно', 'подруг', 'я', 'человек', 'толстокожий']


---
# Получаем из dask.bag очищенный и неочищенный от стоп-слов список со словами

In [ ]:
with ProgressBar():
    bank = b.compute()
    bank_copy = b_copy.compute()

[########################################] | 100% Completed | 126.63 s
[########################################] | 100% Completed | 105.68 s


In [ ]:
print(f"Количество слов без учета стоп-слов: {len(bank):_}\nКоличество слов  с учетом стоп-слов: {len(bank_copy):_}")

Количество слов без учета стоп-слов: 19_530_975
Количество слов  с учетом стоп-слов: 26_634_916


---
# Сравнение полученных списков (без лемматизации)

In [ ]:
bankdict = Counter(bank)
bank_copydict = Counter(bank_copy)

print(f"Самые популярные слова\n{'-'*100}\n|{'стоп-слова не учитываются:':^49}|{'стоп-слова учитываются:':^48}|\n{'-'*100}")

prnt = [print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(bankdict.most_common(15), bank_copydict.most_common(15))]

Самые популярные слова
----------------------------------------------------------------------------------------------------
|           стоп-слова не учитываются:            |            стоп-слова учитываются:             |
----------------------------------------------------------------------------------------------------
|           это                    115299         |           не                    508815         |
----------------------------------------------------------------------------------------------------
|          время                   38249          |           на                    458373         |
----------------------------------------------------------------------------------------------------
|         сказал                   35970          |          что                    329424         |
----------------------------------------------------------------------------------------------------
|         который                  28646          |           он    

In [ ]:
print(f"Самые НЕпопулярные слова\n{'-'*100}\n|{'стоп-слова не учитываются:':^49}|{'стоп-слова учитываются:':^48}|\n{'-'*100}")

prnt = [print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(sorted(bankdict.items(), key=lambda x: x[1])[:15], sorted(bank_copydict.items(), key=lambda x: x[1])[:15])]

Самые НЕпопулярные слова
----------------------------------------------------------------------------------------------------
|           стоп-слова не учитываются:            |            стоп-слова учитываются:             |
----------------------------------------------------------------------------------------------------
|          фрин                      1            |          фрин                     1            |
----------------------------------------------------------------------------------------------------
|   малочувствительной               1            |   малочувствительной              1            |
----------------------------------------------------------------------------------------------------
|          вангу                     1            |         вангу                     1            |
----------------------------------------------------------------------------------------------------
|         невещих                    1            |        невещих

---
# Оптимизирую получение лемм и стемм слов

### Получаю уникальные слова

In [ ]:
unique_bank_without = set(bank)
unique_stopwords = set(bank_copy) - unique_bank_without

In [ ]:
print(f"Уникальных слов  с учетом стоп-слов: {len(unique_stopwords) + len(unique_bank_without)}\nУникальных слов без учета стоп-слов: {len(unique_bank_without)}\n{'Уникальных стоп-слов: ':>37}{len(unique_stopwords)}")

Уникальных слов  с учетом стоп-слов: 686763
Уникальных слов без учета стоп-слов: 686620
               Уникальных стоп-слов: 143


### Применяю к полученным словам функции лемматизации и стемминга

In [ ]:
start = perf_counter()

lemma_dict_without = {word: lemmatize_word(word) for word in unique_bank_without}

lemma_dict_with = {word: lemmatize_word(word) for word in unique_stopwords}
lemma_dict_with.update(lemma_dict_without)
print(f"Время на создание  лемм уникальных слов (секунды): {np.around(perf_counter() - start)}")



start = perf_counter()
stemma_dict_without = {word: stem_word(word) for word in unique_bank_without}

stemma_dict_with = {word: stem_word(word) for word in unique_stopwords}
stemma_dict_with.update(stemma_dict_without)
print(f"Время на создание стемм уникальных слов (секунды): {np.around(perf_counter() - start)}")

Время на создание  лемм уникальных слов (секунды): 169.0
Время на создание стемм уникальных слов (секунды): 46.0


---
## Получаю леммы, стеммы

In [ ]:
lemme_with = map(lambda x: lemma_dict_with[x], bank_copy)


dictlemme_with = Counter(lemme_with)
dictlemme_without = dict(filter(lambda tpl: tpl[0] not in SWrus, dictlemme_with.items()))

In [ ]:
stemme_with = map(lambda x: stemma_dict_with[x], bank_copy)


dictstemme_with = Counter(stemme_with)
dictstemme_without = dict(filter(lambda tpl: tpl[0] not in SWrus, dictstemme_with.items()))

---
# Сравнение полученных списков (стоп-слова не учитываются)

In [ ]:
print(f"Самые популярные слова (стоп-слова не учитываются)\n{'-'*100}\n|{'до лемматизации:':^49}|{'после лемматизации:':^48}|\n{'-'*100}")


prnt = [print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(bankdict.most_common(15), sorted(dictlemme_without.items(), key=lambda x: x[1], reverse=True))]

Самые популярные слова (стоп-слова не учитываются)
----------------------------------------------------------------------------------------------------
|                до лемматизации:                 |              после лемматизации:               |
----------------------------------------------------------------------------------------------------
|           это                    115299         |           я                     275014         |
----------------------------------------------------------------------------------------------------
|          время                   38249          |          это                    183838         |
----------------------------------------------------------------------------------------------------
|         сказал                   35970          |          свой                   174712         |
----------------------------------------------------------------------------------------------------
|         который                  28646

In [ ]:
print(f"Самые НЕпопулярные слова\n{'-'*100}\n|{'до лемматизации:':^49}|{'после лемматизации:':^48}|\n{'-'*100}")


prnt = [print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|\n{'-'*100}") for v1, v2 in zip(sorted(bankdict.items(), key=lambda x: x[1])[:15], sorted(dictlemme_without.items(), key=lambda x: x[1]))]

Самые НЕпопулярные слова
----------------------------------------------------------------------------------------------------
|                до лемматизации:                 |              после лемматизации:               |
----------------------------------------------------------------------------------------------------
|          фрин                      1            |        невещий                    1            |
----------------------------------------------------------------------------------------------------
|   малочувствительной               1            |     киноискусство                 1            |
----------------------------------------------------------------------------------------------------
|          вангу                     1            |       прошмонал                   1            |
----------------------------------------------------------------------------------------------------
|         невещих                    1            |      перевалят

In [ ]:
lemmatize_word("фрин"), lemmatize_word("вангу"), lemmatize_word("нинкино"), dictlemme_without["фрина"]

('фрина', 'ванг', 'нинкин', 10)

---
# Сравнение полученных списков (стоп-слова учитываются)

In [ ]:
print(f"Самые популярные слова (стоп-слова учитываются)\n{'-'*199}\n|{'до лемматизации (стоп-слова есть):':^49}|{'после лемматизации (стоп-слова есть):':^48}|||{'до лемматизации (стоп-слов нет):':^49}|{'после лемматизации (стоп-слов нет):':^48}|\n{'-'*199}")


prnt = [print(f"|{v1[0]:^25}{v1[1]:^24}|{v2[0]:^24}{v2[1]:^24}|||{v3[0]:^24}{v3[1]:^24}|{v4[0]:^24}{v4[1]:^24} |\n{'-'*199}") for v1, v2, v3, v4 in zip(bank_copydict.most_common(15), sorted(dictlemme_with.items(), key=lambda x: x[1], reverse=True), bankdict.most_common(15), sorted(dictlemme_without.items(), key=lambda x: x[1], reverse=True))]

Самые популярные слова (стоп-слова учитываются)
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|       до лемматизации (стоп-слова есть):        |     после лемматизации (стоп-слова есть):      |||        до лемматизации (стоп-слов нет):         |      после лемматизации (стоп-слов нет):       |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|           не                     508815         |           он                    579647         |||          это                    115299         |           я                     275014          |
----------------------------------------------------------------------------------------------------------------------------------------------------